In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime,timedelta
import json
import re

In [183]:
#根据log_index，增量读取log内容
def load_log(file_name, log_index):
    log_value_list = list()
    date_list = list()
    try:
        with open(file_name,'rb') as f:
            for index,content in enumerate(f):
                if index == log_index:
                    log_data = json.loads(content.decode('utf8'))
                    log_data['@timestamp'] = str(datetime.strptime(log_data['@timestamp'][0:19],'%Y-%m-%dT%H:%M:%S'))
                    log_value_list.append(log_data)
                    date_list.append(log_data['@timestamp'])
                    log_index += 1
    except Exception as e:
        print(e)
    #将文本内容转化为pandas的dataframe
    #temp_log = pd.DataFrame(log_value_list)
    return  log_value_list,log_index

In [208]:
time_step = 240 #每x秒读取一次log文件
time_window = 240 #秒,时间窗口
#ip_dict = dict() #ip与对应操作在矩阵的下标
#ip_reserve_list = list() #操作矩阵下标对应的ip
#all_log = pd.DataFrame() #所有的日志数据
#all_url_list = list()
log_index = 0 #记录读到第几行了

increment_log_data, log_index = load_log('0713_access_sdsl_shunnengnet_com_json.log', 0)

In [209]:
#截取url，保留其访问传参方式（POST/GET/PUT/DELETE）,保留url的头与尾
def deal_url(url):
    url_str = url.replace(' HTTP/1.1','')
    if url_str.find(' ') > -1:
        url_type = url_str[0 : url_str.find(' ')+1]
    if url_str.find('/') > -1 and url_str.find('?') > -1:
        url_str = url_type + url_str[url_str.find('/')+1 : url_str.find('?')]
    if url_str.find('/') > -1 and url_str.rfind('/') > -1:
        url_str = url_str[0 : url_str.find('/')] + '/.../' + url_str[url_str.rfind('/')+1 : ]
    return url_str

In [210]:
#根据日志内容，创建一个信息的op_detail
def create_op_detail(row_log):
    op_detail = dict()
    op_detail['start_time'] = row_log['@timestamp']
    op_detail['end_time'] = row_log['@timestamp']
    op_detail['urls'] = list()
    op_detail['urls'].append(deal_url(row_log['request']))
    return op_detail

In [211]:
#传入开始结束时间的字符串(yyyy-MM-dd HH:mm:ss)，计算时间差并返回时差(秒)
def cacl_time_difference(start_time_str, end_time_str):
    start_time = datetime.strptime(start_time_str,'%Y-%m-%d %H:%M:%S')
    end_time = datetime.strptime(end_time_str,'%Y-%m-%d %H:%M:%S')
    return int((end_time - start_time).seconds)

In [212]:
#读取log，生成携带用户行为的复合型字典
def create_user_behavior_dict(increment_log_data, user_behavior_dict, time_window):
    
    for x in increment_log_data:
        if x['sip'] != '':
            
            if x['sip'] not in user_behavior_dict.keys():
                user_behavior_dict[x['sip']] = list()

            if len(user_behavior_dict[x['sip']]) == 0: #opreation内没有op_detail,则创建op_detail
                user_behavior_dict[x['sip']].append(create_op_detail(x))

            if len(user_behavior_dict[x['sip']]) > 0: #opreation内已存在op_detail，此时需要循环该list的所有dict，比较时间差，如果两者时间相差<120s，更新结束时间，添加url；否则继续遍历比较直至找不到合适的区间为止，若找不到合适区间，新建
                new_opreation = 1
                length = len(user_behavior_dict[x['sip']])
                i = 0
                for op_detail in user_behavior_dict[x['sip']]:
                    time_difference = cacl_time_difference(op_detail['start_time'], x['@timestamp']) #如果该操作与本组的第一次操作相差超过time_window
                    if time_difference < time_window:
                        op_detail['end_time'] = x['@timestamp']
                        op_detail['urls'].append(deal_url(x['request']))
                        new_opreation = 0
                        break
                    if new_opreation == 1 and i+1 == length:
                        user_behavior_dict[x['sip']].append(create_op_detail(x))
                        break
                        
                    i += 1
                    
                
    return user_behavior_dict
        
        

In [213]:
user_behavior_dict = dict()

In [214]:
start_time = datetime.now()
user_behavior_dict = create_user_behavior_dict(increment_log_data, user_behavior_dict, time_window)
end_time = datetime.now()
print((end_time - start_time).seconds)

114


In [215]:
i = 0
for x in user_behavior_dict:
    print (user_behavior_dict[x],'\n\n')
    i += 1
    if i>10:
        break

[{'start_time': '2017-07-13 06:25:02', 'end_time': '2017-07-13 06:25:32', 'urls': ['POST /.../getDept.do', 'POST /.../getDept.do', 'GET common/.../scheduleByDept.do', 'POST /.../scheduleByDept.do', 'GET common/.../arrangementList.do', 'GET common/.../arrangementList.do', 'POST /.../getTimeList.do', 'GET common/.../reserveConfirm.do', 'POST /.../reserveConfirm.do', 'GET common/.../patientList.do', 'GET /.../patientinfo.do', 'GET common/.../reserveConfirm.do', 'POST /.../reserveConfirm.do', 'POST /.../reserve.do']}, {'start_time': '2017-07-13 07:26:36', 'end_time': '2017-07-13 07:30:33', 'urls': ['GET /.../noticePage.html', 'GET /.../appointGuide.html', 'GET common/.../patientinfo.do', 'GET common/.../login.do', 'GET /.../register.do', 'GET user/.../getSmsCode.do', 'GET user/.../getSmsCode.do', 'GET user/.../getSmsCode.do', 'GET common/.../notice.do', 'POST common/.../chooseHospital.do', 'GET common/.../chooseExpert.do', 'GET common/.../expertNotice.do', 'POST common/.../deptList.do', 'P

In [216]:
#定义模板list    这个现在应该用不到了吧

templates = list()
templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../deptList.do','POST /.../getDept.do','GET /common/.../scheduleByDept.do','POST /.../scheduleByDept.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do']) #普通专家预约
templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../expertNotice.do','POST /common/.../deptList.do','POST /.../getDept.do','GET /common/.../scheduleByDept.do','POST /.../scheduleByDept.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do'])#知名专家预约
templates.append(['GET /common/.../notice.do','POST /common/.../chooseHospital.do','GET /common/.../chooseExpert.do','GET /common/.../specialClinic.do','POST /.../getDepts.do','GET /common/.../specialSchedule.do','POST /.../specialSchedule.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do','GET /common/.../reserveConfirm.do','POST /.../reserveConfirm.do','POST /.../reserve.do'])#特色门诊预约
templates.append(['GET /common/.../patientinfo.do','GET /common/.../reserveList.do','POST /.../getReserveList.do','GET /common/.../reserveDetail.do','POST /.../cancelReserve.do'])#取消预约
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /.../patientinfo.do','GET /common/.../addPatient.do','POST /.../checkIdCard.do','POST /.../create.do'])#新增就诊人
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','POST /.../setDefault.do',''])#设置默认就诊人
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','GET /common/.../bind.do','POST /.../bindHosCard.do'])#绑卡
templates.append(['GET /common/.../patientinfo.do','GET /common/.../patientMgr.do ','GET /common/.../editPatient.do','POST /.../getPatientInfoById.do','GET /.../deletePatient.do'])#删除就诊人
templates.append(['GET /.../init.do','GET /common/.../reportType.do ','GET /common/.../reportList.do','GET /.../getReportList.do','GET /common/.../reportDetail.do','GET /.../getReportDeatil.do'])#门诊报告
templates.append(['GET /.../init.do','GET /common/.../reportType.do ','GET /common/.../reportList.do','GET /.../getReportInList.do','GET /.../getReportDeatil.do'])#住院报告
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../balance.do','GET /.../patientBalance.do'])#门诊预交金
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../hosBalance.do','POST /.../inBalance.do'])#住院预交金
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../outpFeeList.do','POST /.../feeList.do','GET /common/.../feeDetail.do','POST /.../feeDetail.do'])#门诊费用
templates.append(['GET /.../init.do','GET /common/.../index.do','GET /common/.../inpFeeList.do','POST /.../feeList.do','GET /common/.../feeDetail.do','POST /.../feeDetail.do'])#住院费用
templates.append(['GET /.../init.do','GET /common/.../reserveQueueList.do','POST /.../getQueueList.do'])#候诊队列
templates.append(['POST coreServlet'])#处理微信发送来的数据
templates.append(['GET common/.../arrangementList.do', 'GET /.../404.do','POST /.../getTimeList.do'])#异常模板1
#templates.append(['POST /.../getTimeList.do','POST /.../getTimeList.do','GET common/.../arrangementList.do','GET /.../404.do','GET common/.../arrangementList.do','GET /.../404.do'])#异常模板1
templates.append(['GET common/.../scheduleByDept.do','POST /.../scheduleByDept.do','POST /.../getTimeList.do','GET common/.../arrangementList.do','POST /.../getTimeList.do', 'POST /.../getTimeList.do','POST /.../getTimeList.do','POST /.../getTimeList.do', 'POST /.../getTimeList.do'])#异常模板2
#templates.append(['GET common/.../arrangementList.do', 'GET common/.../arrangementList.do', 'GET common/.../arrangementList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do', 'POST /.../getTimeList.do'])#异常模板3
templates.append(['GET /common/.../deptList.do','POST /.../getDept.do','GET /common/.../scheduleByDept.do','POST /.../scheduleByDept.do','GET /common/.../arrangementList.do','POST /.../getTimeList.do'])

In [217]:
#!!!生成出现过的ip列表unique_dict
#为每一个ip计算他访问过多少唯一页面，占访问次数的比例。frequency这英文似乎给错了，不是频率，而是覆盖率coverage 

unique_dict = dict()
for x in increment_log_data:
    ip = x['sip']
    if ip not in unique_dict.keys():
        unique_dict[ip] = dict()
        unique_dict[ip]['coverage'] = 0
        unique_dict[ip]['urls'] = list()
    url = deal_url(x['request'])
    unique_dict[ip]['urls'].append(url)
    
for ip in unique_dict.keys():
    unique_list = list()
    for url in unique_dict[ip]['urls']:
        if url not in unique_list:
            unique_list.append(url)
    #print(len(unique_dict[ip]['urls']))
    unique_dict[ip]['coverage'] = len(unique_list) / len(unique_dict[ip]['urls'])

In [218]:
def get_operation_label(ip):
    global unique_dict
    class_type = 0
    if unique_dict[ip]['coverage'] < 0.123:
        class_type = 1
    return class_type

In [219]:
exception_ip_list = list()
for x in unique_dict.keys():
    if unique_dict[x]['coverage'] < 0.123:
        exception_ip_list.append(x)
len(exception_ip_list)/len(unique_dict)

0.010169491525423728

In [220]:
get_operation_label('27.204.15.164')

1

In [221]:
#----------------------start
class IPAccessInfo(object):
    def __init__(self, all_url_list, all_url_dict_list):
        self.all_url_list = all_url_list
        self.all_url_dict_list = all_url_dict_list
        
import pickle
fp = open('url_info.pkl','rb',True)
iPAccessInfo = pickle.load(fp)

In [222]:
# from keras.models import Sequential
# from keras.layers import Embedding

# #列出唯一的url
# url_dict = dict()
# for urls in iPAccessInfo.all_url_list:
#     for url in urls:
#         if url not in url_dict.keys():
#             url_dict[url] = 1
#         else:
#             url_dict[url] = url_dict[url] + 1
            
# url_list = list(url_dict.keys())

# #将url embedding，转为固定长度的向量
# num_urls = len(url_list)
# model = Sequential()
# model.add(Embedding(num_urls, 32, input_length=1))
# model.compile('rmsprop', 'mse')

# input_array = np.array(range(num_urls))   #生成一个[num_url,1]的矩阵
# output_array = model.predict(input_array)
# print (output_array.shape)
# print (output_array)

# # 建立url_dict备查
# url_dict = dict()
# idx = 0
# for url in url_list:
#     url_dict[url] = output_array[idx,0,:]
#     idx = idx + 1

In [223]:
time_steps = 20

#老版本数据转换，将按照时间窗口、ip进行归类的urls列表，转化为时序data
# def operations_to_sequence(all_url_dict_list):
#     operation_matrix = np.zeros((len(all_url_dict_list),time_steps,32))
#     ip_operation_index = 0
#     for ip_operations in all_url_dict_list: #对于每小段窗口，以ip归类的urls集合
#         ip_operation_url_index = 0
#         for url in ip_operations['urls']: #如果是dict字典的话
#             url_vector = url_dict[url]
#             operation_matrix[ip_operation_index, ip_operation_url_index, :] = url_vector
#             ip_operation_url_index = ip_operation_url_index + 1
#             if ip_operation_url_index>=20:  #不超过20个长度的序列，可以改
#                 break
#         ip_operation_index = ip_operation_index + 1
#     return operation_matrix


#新版本数据转换，将按照ip地址，然后里面的几段行为数据，转化为时序data
# 数据格式为：
# {'223.104.186.241': [{'end_time': '2017-07-13 06:25:32',
#    'start_time': '2017-07-13 06:25:02',
#    'urls': ['POST /.../getDept.do',
#     'POST /.../getDept.do',
#     'GET common/.../scheduleByDept.do',
#     'POST /.../scheduleByDept.do',
#     'GET common/.../arrangementList.do',
#     'GET common/.../arrangementList.do',
#     'POST /.../getTimeList.do',
#     'GET common/.../reserveConfirm.do',
#     'POST /.../reserveConfirm.do',
#     'GET common/.../patientList.do',
#     'GET /.../patientinfo.do',
#     'GET common/.../reserveConfirm.do',
#     'POST /.../reserveConfirm.do',
#     'POST /.../reserve.do']},
#   {'end_time': '2017-07-13 07:31:18',
#    'start_time': '2017-07-13 07:26:36',
#    'urls': ['GET /.../noticePage.html',

def operations_to_sequence(user_behavior_dict):
    #计算这个dict中有多少段操作
    n_op_details = 0
    for x in user_behavior_dict.keys():
        n_op_details += len(user_behavior_dict[x])
    
    operation_matrix = np.zeros((n_op_details,20,32))
    ip_operation_index = 0
    for ip in user_behavior_dict.keys(): #对于每个ip,得到op_detail列表
        for op_detail in user_behavior_dict[ip]: #对于每一组urls操作
            ip_operation_url_index = 0
            for url in op_detail['urls']: #对于每一个url
                url_vector = url_dict[url]  #翻译为url对应的vector
                operation_matrix[ip_operation_index, ip_operation_url_index, :] = url_vector
                ip_operation_url_index += 1
                if ip_operation_url_index>=time_steps:  #不超过20个长度的序列，可以改
                    break
            ip_operation_index += 1
    return operation_matrix

In [224]:
data = operations_to_sequence(user_behavior_dict)
print (data.shape) #数据编码，我就是快，数据量大640倍，还是快。。。

(20711, 20, 32)


In [225]:
n_class = 2
# 旧版的数据转换
# def label_operations(all_url_dict_list):
#     operation_label = np.zeros(len(all_url_dict_list)) # 先按照0 1 2 3，一维数组排序
#     ip_operation_index = 0
#     for ip_operations in all_url_dict_list: #对于每小段窗口，以ip归类的urls集合
#         operation_label[ip_operation_index] = get_operation_label(ip_operations['ip'],ip_operations['time'])
#         ip_operation_index = ip_operation_index + 1
        
#     #转化为one-hot编码的label
#     return operation_label


def label_operations(user_behavior_dict):
    #计算这个dict中有多少段操作
    n_op_details = 0
    for x in user_behavior_dict.keys():
        n_op_details += len(user_behavior_dict[x])
        
    operation_labels = np.zeros(n_op_details) # 先按照0 1 2 3，一维数组排序
    ip_operation_index = 0
    for ip in user_behavior_dict.keys(): #对于每个ip,得到op_detail列表
        for op_index in range(len(user_behavior_dict[ip])): #对于每一组urls操作，得到其index 
            operation_labels[ip_operation_index] = get_operation_label(ip)
            ip_operation_index += 1
            
    return operation_labels

In [226]:
labels = label_operations(user_behavior_dict)
print (labels.shape)

(20711,)


In [227]:
# #labels
# b = np.argsort(labels)
# b = b.tolist()
# b.reverse()
# for x in b:
#     if labels[x] > 0.9 and labels[x] < 1:
#         #ip = iPAccessInfo.all_url_dict_list[x]['ip']
#         print(iPAccessInfo.all_url_dict_list[x])
        
        

In [228]:
(labels==1).sum() #验证每个类别的个数

2204

In [229]:
#网址embedding表示，应该不需要进行正规化吧
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

# #由于时序序列数据，是（？，时间长度，数据长度）的，所以应该执行一个reshape
# X = scaler.fit_transform(data.reshape([-1,20*32]))
# X = X.reshape([-1,20,32])

In [230]:
# 定义保存数据的类
import numpy as np

#定义类
class IPAccessData(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
#         self.ip_dict = ip_dict
#         self.ip_reserve_list = ip_reserve_list
#         self.data_scaler = data_scaler

# 保存文件为pkl
import pickle

iPAccessData = IPAccessData(data,labels)
fp = open('iPAccessData_time_window_'+str(time_window)+'.pkl','wb',True)
pickle.dump(iPAccessData, fp)

In [231]:
time_window

240